In [6]:
# https://www.geeksforgeeks.org/k-means-clustering-introduction/
import random

def ReadData(fileName):

    # Read the file, splitting by lines
    f = open(fileName, 'r')
    lines = f.read().splitlines()
    f.close()
    items = []
    for i in range(1, len(lines)):
        line = lines[i].split(',')
        itemFeatures = []
        for j in range(len(line)-1):
            # Convert feature value to float
            v = float(line[j])
            # Add feature value to dict
            itemFeatures.append(v)
        items.append(itemFeatures)
    random.shuffle(items)
    return items

items = ReadData("k_means.txt")
print(items)

[[6.7, 3.1, 4.4, 1.4], [6.8, 3.0, 5.5, 2.1], [6.7, 3.0, 5.2, 2.3], [6.3, 2.5, 4.9, 1.5], [4.4, 3.2, 1.3, 0.2], [5.8, 2.7, 3.9, 1.2], [5.3, 3.7, 1.5, 0.2], [6.5, 2.8, 4.6, 1.5], [6.0, 3.4, 4.5, 1.6], [5.6, 2.9, 3.6, 1.3], [6.3, 3.4, 5.6, 2.4], [6.8, 2.8, 4.8, 1.4], [5.8, 2.8, 5.1, 2.4], [4.8, 3.1, 1.6, 0.2], [6.1, 2.8, 4.7, 1.2], [5.7, 2.9, 4.2, 1.3], [5.6, 3.0, 4.1, 1.3], [5.0, 3.5, 1.6, 0.6], [5.5, 3.5, 1.3, 0.2], [6.3, 2.9, 5.6, 1.8], [5.8, 2.7, 5.1, 1.9], [5.5, 2.3, 4.0, 1.3], [5.4, 3.9, 1.3, 0.4], [4.8, 3.0, 1.4, 0.3], [4.9, 2.5, 4.5, 1.7], [5.7, 2.6, 3.5, 1.0], [4.3, 3.0, 1.1, 0.1], [6.3, 2.7, 4.9, 1.8], [5.0, 3.6, 1.4, 0.2], [5.9, 3.0, 4.2, 1.5], [6.9, 3.1, 4.9, 1.5], [6.8, 3.2, 5.9, 2.3], [6.7, 3.0, 5.0, 1.7], [5.2, 4.1, 1.5, 0.1], [6.9, 3.1, 5.4, 2.1], [5.5, 2.4, 3.7, 1.0], [5.1, 3.8, 1.6, 0.2], [5.0, 3.2, 1.2, 0.2], [7.6, 3.0, 6.6, 2.1], [7.1, 3.0, 5.9, 2.1], [5.7, 3.8, 1.7, 0.3], [5.5, 4.2, 1.4, 0.2], [5.7, 3.0, 4.2, 1.2], [6.3, 2.5, 5.0, 1.9], [6.0, 2.7, 5.1, 1.6], [6.4, 3.2

In [15]:
import sys

def FindColMinMax(items):
    maxSize = sys.maxsize
    n = len(items[0]);
    minima = [maxSize for i in range(n)]
    maxima = [-maxSize -1 for i in range(n)]

    for item in items:
        for f in range(len(item)):
            if (item[f] < minima[f]):
                minima[f] = item[f]

            if (item[f] > maxima[f]):
                maxima[f] = item[f]

    return minima,maxima

minima, maxima = FindColMinMax(items)
print(minima)
print(maxima)

[4.3, 2.0, 1.0, 0.1]
[7.9, 4.4, 6.9, 2.5]


In [ ]:
def InitializeMeans(items, k, cMin, cMax):

    # Initialize means to random numbers between
    # the min and max of each column/feature
    f = len(items[0]) # number of features
    means = [[0 for i in range(f)] for j in range(k)];

    for mean in means:
        for i in range(len(mean)):

            # Set value to a random float
            # (adding +-1 to avoid a wide placement of a mean)
            mean[i] = uniform(cMin[i]+1, cMax[i]-1)

    return means

means = InitializeMeans(items, )